In [3]:
#import OpenAI key with helper function
from helper import get_openai_api_key

OPENAI_API_KEY = get_openai_api_key()

In [4]:
#A lot of modules use async and we want them to be compatible with Jupyter notebook
import nest_asyncio

nest_asyncio.apply()

In [5]:
import os
import glob

# Define the path to the directory containing the PDF files
folder_path = 'Ehlers-Danlos-1'

# Get the list of all PDF files in the directory
pdf_files = glob.glob(os.path.join(folder_path, '*.pdf'))
print(pdf_files)

# Extract just the filenames (optional)
pdf_filenames = [os.path.basename(pdf) for pdf in pdf_files]

# Print the list of PDF filenames
print(pdf_filenames)


['Ehlers-Danlos-1\\2024_EDS_1.pdf', 'Ehlers-Danlos-1\\2024_EDS_2.pdf', 'Ehlers-Danlos-1\\2024_EDS_3.pdf', 'Ehlers-Danlos-1\\2024_EDS_4.pdf', 'Ehlers-Danlos-1\\2024_EDS_5.pdf', 'Ehlers-Danlos-1\\2024_EDS_6.pdf', 'Ehlers-Danlos-1\\2024_EDS_7.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_1.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_2.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_3.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_4.pdf', 'Ehlers-Danlos-1\\Unknown_EDS_5.pdf']
['2024_EDS_1.pdf', '2024_EDS_2.pdf', '2024_EDS_3.pdf', '2024_EDS_4.pdf', '2024_EDS_5.pdf', '2024_EDS_6.pdf', '2024_EDS_7.pdf', 'Unknown_EDS_1.pdf', 'Unknown_EDS_2.pdf', 'Unknown_EDS_3.pdf', 'Unknown_EDS_4.pdf', 'Unknown_EDS_5.pdf']


In [6]:
from utils import get_doc_tools
from pathlib import Path

# Ensure function names are within the allowed length limit
def truncate_function_name(name, max_length=64):
    return name if len(name) <= max_length else name[:max_length]

paper_to_tools_dict = {}
for pdf in pdf_files:
    print(f"Getting tools for paper: {pdf}")
    vector_tool, summary_tool = get_doc_tools(pdf, Path(pdf).stem)
    #vector_tool, summary_tool = get_doc_tools(pdf, truncate_function_name(Path(pdf).stem))
    paper_to_tools_dict[pdf] = [vector_tool, summary_tool]
    #print(vector_tool)
    #print(summary_tool)

Getting tools for paper: Ehlers-Danlos-1\2024_EDS_1.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_2.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_3.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_4.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_5.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_6.pdf
Getting tools for paper: Ehlers-Danlos-1\2024_EDS_7.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_1.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_2.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_3.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_4.pdf
Getting tools for paper: Ehlers-Danlos-1\Unknown_EDS_5.pdf


In [7]:
all_tools = [t for pdf in pdf_files for t in paper_to_tools_dict[pdf]]
#all_tools = [truncate_function_name(tool) for tool in all_tools]


In [8]:
# define an "object" index and retriever over these tools
from llama_index.core import VectorStoreIndex
from llama_index.core.objects import ObjectIndex

obj_index = ObjectIndex.from_objects(
    all_tools,
    index_cls=VectorStoreIndex,
)

In [9]:
obj_retriever = obj_index.as_retriever(similarity_top_k=3)


In [10]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")

In [11]:
from llama_index.core.agent import FunctionCallingAgentWorker
from llama_index.core.agent import AgentRunner

agent_worker = FunctionCallingAgentWorker.from_tools(
    tool_retriever=obj_retriever,
    llm=llm, 
    verbose=True
)
agent = AgentRunner(agent_worker)

In [1]:

response = agent.query(
    "Do people with EDS suffer from dislocations, and if so, how do they manifest?"
)
print(str(response))

NameError: name 'agent' is not defined

: 